In [5]:
import pandas as pd
import pandas as pd
import plotly.express as px
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


df_income = pd.read_excel("./new_data/2024_exp.xlsx", sheet_name="Income", skiprows=1)
df_expenses = pd.read_excel("./new_data/2024_exp.xlsx", sheet_name="Expenses",skiprows=1)

In [6]:
df_expenses

,Date and time,Category,Account,Amount in default currency,Default currency,Amount in account currency,Account currency,Transaction amount in transaction currency,Transaction currency,Tags,Comment
0,2024-09-29,Cafe,Main,23.71,CAD,23.71,CAD,NaN,NaN,NaN,NaN
1,2024-09-25,Groceries,Main,69.55,CAD,69.55,CAD,NaN,NaN,NaN,NaN
2,2024-09-24,Cafe,Main,34.28,CAD,34.28,CAD,NaN,NaN,NaN,NaN
3,2024-09-23,Cafe,Main,23.96,CAD,23.96,CAD,NaN,NaN,NaN,NaN
4,2024-09-22,Cafe,Main,20.03,CAD,20.03,CAD,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
242,2024-01-06,Groceries,Main,23.71,CAD,23.71,CAD,NaN,NaN,NaN,NaN
243,2024-01-05,Cafe,Main,29.07,CAD,29.07,CAD,NaN,NaN,NaN,Beers for friends
244,2024-01-02,Haircut,Main,20.34,CAD,20.34,CAD,NaN,NaN,NaN,NaN
245,2024-01-01,Cafe,Main,16.94,CAD,16.94,CAD,NaN,NaN,NaN,NaN


In [7]:
# Droping some columns that were unnecessary..
col_to_drop = [
    "Transaction amount in transaction currency",
    "Transaction currency",
    "Amount in account currency",
    "Account currency"
]
df_income.drop(columns=col_to_drop, inplace=True)
df_expenses.drop(columns=col_to_drop, inplace=True)


# adding type to each df 
df_income["Type"] = "Income"
df_expenses["Type"] = "Expenses"

In [9]:
# Joining these two dataframes of expenses and income into one 
df = pd.concat([df_income, df_expenses], ignore_index=True)
# Renaming some columns for better understanding and prediciton
df = df.rename(columns={
    'Date and time': 'Date', 
    'Amount in default currency': 'Amount',
    'Default currency' : 'Currency'
    })
df

,Date,Category,Account,Amount,Currency,Tags,Comment,Type
0,2024-09-27,Paycheck,Main,149.00,CAD,NaN,NaN,Income
1,2024-09-26,Balancing,Main,8.00,CAD,NaN,NaN,Income
2,2024-08-30,Paycheck,Main,300.95,CAD,NaN,NaN,Income
3,2024-08-29,Balancing,Main,1.00,CAD,NaN,NaN,Income
4,2024-08-26,Cashback,Main,26.00,CAD,NaN,NaN,Income
...,...,...,...,...,...,...,...,...
279,2024-01-06,Groceries,Main,23.71,CAD,NaN,NaN,Expenses
280,2024-01-05,Cafe,Main,29.07,CAD,NaN,Beers for friends,Expenses
281,2024-01-02,Haircut,Main,20.34,CAD,NaN,NaN,Expenses
282,2024-01-01,Cafe,Main,16.94,CAD,NaN,NaN,Expenses


In [10]:


# 1. Data Cleaning
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# # Drop 'Comment' column if not relevant
# df.drop('Comment', axis=1, inplace=True)

fig = px.line(df, x='Date', y='Amount', color='Type', title='Income and Expenses Over Time')
fig.update_layout(width=1080, height=700)  # Set width and height
fig.show()

# Plot distribution of expenses across categories
fig = px.histogram(df[df['Type'] == 'Expenses'], x='Category', y='Amount', title='Distribution of Expenses Across Categories')
fig.update_layout(width=1080, height=700)  # Set width and height
fig.show()

